# Creating a Corpus for German Political Speeches

The vector space model (VSM) is a common representation for documents in order to perfom clustering, topic modeling, classification, similarity search etc.

In this case, we want to represent documents as bag-of-words. Therefore, the documents in the input text files (CSV, one document per line) are tokenized and converted into a corpus of indexed terms/tokens.

### Pre-processing Steps

The input data is usually messy. But instead of extensive pre-processing, e.g. cleaning of markup, punctuation, etc., we will simply extract all alphabetic sequences as tokens and nomalize them including following steps:

  * convert to lower case
  * remove stopwords
  * create n-grams
  * **`TODO:`** stemming
  * **`TODO:`** lemmatization

### References

* [Tutorial on Corpora and Vector Spaces](https://radimrehurek.com/gensim/tut1.html) from [Gensim](https://radimrehurek.com/gensim/index.html).
* [Tutorial on Topic Modeling with Gensim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) from [Machine Learning Plus](https://www.machinelearningplus.com/).

## Prerequisites

### Libraries and Constants

In [ ]:
from collections import defaultdict
from pprint import pprint
import pandas as pd
import string
import os
import re
import time

# input files
data_dir        = '../data/'
filename        = data_dir + 'Bundesregierung.csv'

# output files
corpus_dir      = '../corpus/'
dict_filename   = corpus_dir + 'gps_ngrams.dict'
corpus_filename = corpus_dir + 'gps_ngrams_bow.mm'

# ensure output directory exists
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

### Helper Functions

In [ ]:
def print_diff(start_time):
    print("--- took %d:%.2d minutes ---" % divmod(time.time() - start_time, 60))

def most_frequent(tokens, topn=10):
    frequency = defaultdict(int)
    for doc in tokens:
        for term in doc:
            frequency[term] += 1
    return sorted(frequency.items(), key=lambda t: t[1], reverse=True)[0:topn]

## Read the Documents

In [ ]:
start_time = time.time()
df = pd.read_csv(filename)
print_diff(start_time)

print(len(df), 'documents imported')
df.head()

## Analyse Compound Words

We want to recognize these compound words later in the n-gram detection.

In [ ]:
PAT_COMPOUND = re.compile(r'\w+[-]\w+')

compounds = df['text'].apply(lambda doc: [match.group() for match in PAT_COMPOUND.finditer(doc)])
pprint(most_frequent(compounds, 10))

### Tokenize

In [ ]:
from gensim.utils import tokenize

def tokens(documents):
    """Convert all documents into a list of lowercase tokens using Gensim's tokenize() function."""
    return [tokenize(doc, lower=True) for doc in documents]

# explicit tokenization
start_time = time.time()
tokens = [[t for t in tokenize(doc, lower=True)] for doc in df['text']]
print_diff(start_time)

pprint(" ".join(tokens[0][0:100]))

## N-Grams

In [ ]:
from gensim.models.phrases import Phrases, Phraser

print('Building bigrams...')
start_time = time.time()
bigram_model = Phrases(tokens, min_count=1, threshold=10)
print_diff(start_time)

print(bigram_model)

print('Building trigrams...')
start_time = time.time()
bigrams = list(bigram_model[tokens])
trigram_model = Phrases(bigrams, min_count=1, threshold=10)
print_diff(start_time)

print(trigram_model)

print('Optimizing bigram/trigram models...')
# optimize bigram, trigram models
start_time = time.time()
bigram_model = Phraser(bigram_model)
trigram_model = Phraser(trigram_model)
print_diff(start_time)

In [ ]:
def n_grams(documents):
    return trigram_model[list(bigram_model[documents])]

freq = defaultdict(int)

start_time = time.time()
for doc in n_grams(tokens):
    for word in doc:
        if '_' in word:
            freq[word] += 1
print_diff(start_time)

freq_desc = sorted(freq.items(), key=lambda t: t[1], reverse=True)

In [ ]:
for i, kv in enumerate(freq_desc[0:10]):
    print('{}. {}'.format(i+1, kv))

In [ ]:
for doc in n_grams(tokens[0:2]):
    print(doc[0:30])

## Load Stopwords

Possible stopword sources:
  * nltk (231)
  * https://github.com/stopwords-iso/stopwords-de (621)
  * https://github.com/solariz/german_stopwords (1855)

In [ ]:
stopwords_filename = '../data/stopwords-de.txt'

with open(stopwords_filename) as f:
    stopwords = f.read().splitlines()
    
#add more stopwords
stopwords.extend('anbelangt anstelle bisschen dinge dingen finde frau geehrter geehrte gerne hierbei insofern inzwischen jedenfalls lediglich liebe könnten manchmal möglichst nunmehr offensichtlich punkt selber sicherlich sogar sozusagen vieles weiterhin wiederum zuletzt'.split())

stopwordsdict = dict.fromkeys(stopwords, 1)

print(len(stopwordsdict), "stopwords")

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_de = stopwords.words('german')


print(len(stopwords_de))
stopwords_de

for k in stopwords_de:
    #print(k)
    if k not in stopwordsdict:
        print(k)

#stopwordsdict.update(dict.fromkeys(stopwords_de, 1))
#print(len(stopwordsdict), "stopwords")

In [ ]:
def remove_stopwords(tokens):
    return [[word for word in doc if word not in stopwordsdict] for doc in tokens]

In [ ]:
tokens_nostop = remove_stopwords(tokens)
tokens_ngram  = n_grams(tokens_nostop)

print(tokens_ngram[0][0:50])

In [ ]:
ngrams = [[word for word in doc if '_' in word] for doc in tokens_ngram]

pprint(most_frequent(ngrams, 10))

# Create Dictionary

In [ ]:
from gensim import corpora

print('Creating Dictionary...')

start_time = time.time()
dictionary = corpora.Dictionary(tokens_ngram)
print_diff(start_time)

print(dictionary)

In [ ]:
dfs_desc = sorted(dictionary.dfs.items(), key=lambda t: t[1], reverse=True)

print('--- Most Frequent Tokens in X Documents', dictionary.num_docs, 'Documents ---')
for (k,v) in dfs_desc[0:10]: print('{freq}: {token}'.format(token=dictionary[k], freq=v))

print('--- Least Frequent Tokens in X Documents', dictionary.num_docs, 'Documents ---')
for (k,v) in dfs_desc[-10:]: print('{freq}: {token}'.format(token=dictionary[k], freq=v))

### Filter extreme tokens

* tokens which occur in more than 30% of all documents.
* tokens which occur in less than 5 documents.

In [ ]:
print('Filtering extreme tokens')
freq_before = len(dictionary)
dictionary.filter_extremes(no_below=5, no_above=0.3)
print('{} token before -> {} after'.format(freq_before, len(dictionary)))

In [ ]:
dfs_desc = sorted(dictionary.dfs.items(), key=lambda t: t[1], reverse=True)

print('--- Most Frequent Token Occurrences in', dictionary.num_docs, 'Documents ---')
for (k,v) in dfs_desc[0:10]: print('{freq}: {token}'.format(token=dictionary[k], freq=v))

print('--- Least Frequent Token Occurrences in', dictionary.num_docs, 'Documents ---')
for (k,v) in dfs_desc[-10:]: print('{freq}: {token}'.format(token=dictionary[k], freq=v))

### Saving Dictionary

In [ ]:
print('Saving Dictionary to', dict_filename)
start_time = time.time()
dictionary.save(dict_filename)
print_diff(start_time)

# Create Corpus

In [ ]:
print('Creating Corpus')
start_time = time.time()
corpus_bow = [dictionary.doc2bow(doc) for doc in tokens_ngram]
print_diff(start_time)

In [ ]:
print('Saving Corpus to', corpus_filename)
start_time = time.time()
corpora.MmCorpus.serialize(corpus_filename, corpus_bow)
print_diff(start_time)